*this markdown cell loads the flot jQuery plugin, used for plotting*

*CEK: and defines a widget that takes json input and uses a callback to modify Python objects*

In [1]:
%%html
<script type="text/javascript">
    // load flot
    $.getScript("https://cdnjs.cloudflare.com/ajax/libs/flot/0.7/jquery.flot.min.js", function(){});
    
    // define the widgets
    update_plot_js = function(msg_type, content){
        // callback for updating the plot with the output of the request
        if (msg_type !== 'display_data')
            return;
        var plotObject = content['data']['application/json'];
        if (plotObject != undefined){
            plotObject = JSON.parse(plotObject);
            $.plot($("#"+plotObject.objectName+"_"+plotObject.widgetNumber), plotObject.lines);
        } else {
            console.log("no lines?!");
            console.log(data);
        }
    };
    
    update = function(objectName,widgetNumber){
        var kernel = IPython.notebook.kernel;
        if (!kernel) return;
        // execute update on the kernel
        var n = $('div#n_slider_'+objectName+'_'+widgetNumber).slider("value");
        $('span#n_label_'+objectName+'_'+widgetNumber).text("n = " + n);

        var xmax = $('div#xmax_slider_'+objectName+'_'+widgetNumber).slider("value");
        $('span#xmax_label_'+objectName+'_'+widgetNumber).text("xmax = " + xmax);

        var npoints = $('div#npoints_slider_'+objectName+'_'+widgetNumber).slider("value");
        $('span#npoints_label_'+objectName+'_'+widgetNumber).text("npoints = " + npoints);

        var args = n + ", xmax=" + xmax + ", npoints=" + npoints;
        kernel.execute(objectName+".update_plot_py(" + args + ")", {'output': update_plot_js});
    };
    
    init = function(objectName,widgetNumber){
    $('div#n_slider_'+objectName+'_'+widgetNumber).slider({
        min : 1,
        max : 20,
        value : 4,
        slide : request_update,
        change: request_update
    });
    $('div#xmax_slider_'+objectName+'_'+widgetNumber).slider({
        min : 1,
        max : 32,
        step : 0.2,
        value : 10,
        slide : request_update,
        change: request_update
    });
    $('div#npoints_slider_'+objectName+'_'+widgetNumber).slider({
        min : 2,
        max : 128,
        step : 1,
        value : 100,
        slide : request_update,
        change: request_update
    });
    update(objectName,widgetNumber);
    };
    
    request_update = function(event,ui){
        update($(ui.handle).parent().attr('objectName'),$(ui.handle).parent().attr('widgetNumber'));
    };
</script>

# Interactive JavaScript plots with kernel callbacks

This is an example of a plotting widget, entirely in javascript, 
which can make callbacks to the Kernel to update data.

In this case, there is a user-defined function (`update_plot`) that
generates the new data for the plot.  The widget has sliders, which
trigger calls to this function when they change.  There is a javascript
callback hooked up, which updates a plot area with the new data when it arrives.

You must run this one code cell to define the function before it is available to the widget,
then the plot should appear when you move one of the sliders.

In [2]:
import json
from IPython.core.display import JSON, display,HTML,Javascript
from sympy.functions.special.bessel import jn
from numpy import linspace,array
class PlotObject:
    def __init__(self,myObjectName,n,xmax=5,npoints=10):
        self.objectName=myObjectName
        self.lines =[]
        self.n=n
        self.xmax=xmax
        self.npoints=npoints
        self.widgetNumber = 0
        x = linspace(0, xmax, npoints)
        for i in range(1,n+1):
            self.lines.append(zip(x,array([float(jn(xi,i).n()) for xi in x])))
    def _repr_html_(self):
        self._html_data = """
<div id=%(objectName)s_%(widgetNumber)s style="width:600px;height:300px;"></div>

<span id="n_label_%(objectName)s_%(widgetNumber)s" objectName=%(objectName)s widgetNumber=%(widgetNumber)s></span>
<div id="n_slider_%(objectName)s_%(widgetNumber)s" objectName=%(objectName)s widgetNumber=%(widgetNumber)s></div>

<span id="xmax_label_%(objectName)s_%(widgetNumber)s" objectName=%(objectName)s widgetNumber=%(widgetNumber)s></span>
<div id="xmax_slider_%(objectName)s_%(widgetNumber)s" objectName=%(objectName)s widgetNumber=%(widgetNumber)s></div>

<span id="npoints_label_%(objectName)s_%(widgetNumber)s" objectName=%(objectName)s widgetNumber=%(widgetNumber)s></span>
<div id="npoints_slider_%(objectName)s_%(widgetNumber)s" objectName=%(objectName)s widgetNumber=%(widgetNumber)s></div>
""" % {'objectName':self.objectName,'widgetNumber':self.widgetNumber}
        #print self._html_data
        return self._html_data
    @property
    def html(self):
        return HTML(self._repr_html_())
    def _repr_javascript_(self):
        self._javascript_data = """
    init('%(objectName)s',%(widgetNumber)s);
""" % {'objectName':self.objectName,'widgetNumber':self.widgetNumber}
        #print self._javascript_data
        return self._javascript_data
    @property
    def javascript(self):
        return Javascript(self._repr_javascript_())
    def _repr_json_(self):
        self._json_data = json.dumps(self.__dict__)
    @property
    def json(self):
        return JSON(self._json_data)
    def update_plot_py(self,n, xmax=10, npoints=200):
        x = linspace(0, xmax, npoints)
        self.lines = []
        self.n = n
        self.xmax = xmax
        self.npoints = npoints
        for i in range(1,n+1):
            self.lines.append(zip(x,array([float(jn(xi,i).n()) for xi in x])))
        display(JSON(json.dumps(self.__dict__)))
    def edit(self):
        self.widgetNumber += 1
        display(po.html)
        display(po.javascript)
po = PlotObject("po",5)
po.edit()

<IPython.core.display.Javascript object>

In [3]:
print po.n,po.xmax,po.npoints

4 10 100


In [4]:
po.edit()

<IPython.core.display.Javascript object>

In [5]:
print po.n,po.xmax,po.npoints

4 10 100


<html>
<h1>Flot Examples</h1>
    
<div id="placeholder" style="width:600px;height:600px;"></div>
<P>    
<p>Simple example. You don't need to specify much to get an
       attractive look. Put in a placeholder, make sure you set its
       dimensions (otherwise the plot library will barf) and call the
       plot function with the data. The axes are automatically
       scaled.</p>

<script type="text/javascript">
    var d1 = [];
    for (var i = 0; i < 14; i += 0.5)
        d1.push([i, Math.sin(i)]);

    var d2 = [[0, 3], [4, 8], [8, 5], [9, 13]];

    // a null signifies separate line segments
    var d3 = [[0, 12], [7, 12], null, [7, 2.5], [12, 2.5]];
    
    $.plot($("#placeholder"), [ d1, d2]);
</script>
</html>